# Query Sequence Analysis (RFC)

This notebook focuses on sequence analysis, when presented with a workload schedule / sequence of queries. In an average day to day work activity, particular query patterns can be discerned. This pattern distinction allows us to discern which queries will be susceptible to execution over time, allowing us to know ahead of time which queries will be executed against the database.

## Data Preprocessing

### Module Installation and Importing Libraries

In [43]:
# scipy
import scipy as sc
print('scipy: %s' % sc.__version__)
# numpy
import numpy as np
print('numpy: %s' % np.__version__)
# matplotlib
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot
# pandas
import pandas as pd
print('pandas: %s' % pd.__version__)
# scikit-learn
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
import sklearn as sk
print('sklearn: %s' % sk.__version__)
import math

scipy: 1.1.0
numpy: 1.15.2
pandas: 0.23.4
sklearn: 0.20.0


### Configuration Cell

Tweak parametric changes from this cell to influence outcome of experiment. 
NB: This experiment demonstrates at time  step = 1 (1 minute in advance). Further down in experiment, other timestep results are also featured and evaluated.

In [44]:
#
# Experiment Config
tpcds='TPCDS1' # Schema upon which to operate test
lag=3 # Time Series shift / Lag Step. Each lag value equates to 1 minute. Cannot be less than 1
if lag < 1:
    raise ValueError('Lag value must be greater than 1!')
#
test_split=.2 # Denotes which Data Split to operate under when it comes to training / validation
#
# Forest Config
parallel_degree = 4
n_estimators = 500
max_depth = 7
criterion='gini'

### Read data from file into Pandas Dataframes

In [45]:
#
# Open Data
#rep_hist_snapshot_path = 'C:/Users/gabriel.sammut/University/Data_ICS5200/Schedule/' + tpcds + '/v2/rep_hist_snapshot.csv'
rep_hist_snapshot_path = 'D:/Projects/Datagenerated_ICS5200/Schedule/' + tpcds + '/v2/rep_hist_snapshot.csv'
#
rep_hist_snapshot_df = pd.read_csv(rep_hist_snapshot_path)
#
def prettify_header(headers):
    """
    Cleans header list from unwated character strings
    """
    header_list = []
    [header_list.append(header.replace("(","").replace(")","").replace("'","").replace(",","")) for header in headers]
    return header_list
#
rep_hist_snapshot_df.columns = prettify_header(rep_hist_snapshot_df.columns.values)
#
print(rep_hist_snapshot_df.columns.values)

['SNAP_ID' 'DBID' 'INSTANCE_NUMBER' 'SQL_ID' 'PLAN_HASH_VALUE'
 'OPTIMIZER_COST' 'OPTIMIZER_MODE' 'OPTIMIZER_ENV_HASH_VALUE'
 'SHARABLE_MEM' 'LOADED_VERSIONS' 'VERSION_COUNT' 'MODULE' 'ACTION'
 'SQL_PROFILE' 'FORCE_MATCHING_SIGNATURE' 'PARSING_SCHEMA_ID'
 'PARSING_SCHEMA_NAME' 'PARSING_USER_ID' 'FETCHES_TOTAL' 'FETCHES_DELTA'
 'END_OF_FETCH_COUNT_TOTAL' 'END_OF_FETCH_COUNT_DELTA' 'SORTS_TOTAL'
 'SORTS_DELTA' 'EXECUTIONS_TOTAL' 'EXECUTIONS_DELTA'
 'PX_SERVERS_EXECS_TOTAL' 'PX_SERVERS_EXECS_DELTA' 'LOADS_TOTAL'
 'LOADS_DELTA' 'INVALIDATIONS_TOTAL' 'INVALIDATIONS_DELTA'
 'PARSE_CALLS_TOTAL' 'PARSE_CALLS_DELTA' 'DISK_READS_TOTAL'
 'DISK_READS_DELTA' 'BUFFER_GETS_TOTAL' 'BUFFER_GETS_DELTA'
 'ROWS_PROCESSED_TOTAL' 'ROWS_PROCESSED_DELTA' 'CPU_TIME_TOTAL'
 'CPU_TIME_DELTA' 'ELAPSED_TIME_TOTAL' 'ELAPSED_TIME_DELTA' 'IOWAIT_TOTAL'
 'IOWAIT_DELTA' 'CLWAIT_TOTAL' 'CLWAIT_DELTA' 'APWAIT_TOTAL'
 'APWAIT_DELTA' 'CCWAIT_TOTAL' 'CCWAIT_DELTA' 'DIRECT_WRITES_TOTAL'
 'DIRECT_WRITES_DELTA' 'PLSEXEC_TIME_T

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3018: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Changing Matrix Shapes

Changes dataframe shape, in an attempt to drop all numeric data. Below's aggregated data is done so on:
* SNAP_ID
* INSTANCE_NUMBER
* DBID
* SQL_ID

In [46]:
print("Shape Before Aggregation: " + str(rep_hist_snapshot_df.shape))
#
# Group By Values by SNAP_ID , sum all metrics (for table REP_HIST_SNAPSHOT) and drop all numeric
df = rep_hist_snapshot_df.groupby(['SNAP_ID'])['SQL_ID'].apply(list).reset_index()
#
print("Shape After Aggregation: " + str(df.shape))
print(type(df))
print(df.head(100))

Shape Before Aggregation: (64912, 90)
Shape After Aggregation: (820, 2)
<class 'pandas.core.frame.DataFrame'>
    SNAP_ID                                             SQL_ID
0     28190  [03ggjrmy0wa1w, 06dymzb481vnd, 0aq14dznn91rg, ...
1     28191  [04kug40zbu4dm, 0a08ug2qc1j82, 0a08ug2qc1j82, ...
2     28192  [01tp87bk1t2zv, 01tp87bk1t2zv, 01tp87bk1t2zv, ...
3     28193  [01tp87bk1t2zv, 01tp87bk1t2zv, 01tp87bk1t2zv, ...
4     28194  [01tp87bk1t2zv, 01tp87bk1t2zv, 01tp87bk1t2zv, ...
5     28195  [06g9mhm5ba7tt, 0kcbwucxmazcp, 0kkhhb2w93cx0, ...
6     28196  [06g9mhm5ba7tt, 0kcbwucxmazcp, 0kkhhb2w93cx0, ...
7     28197  [01tp87bk1t2zv, 01tp87bk1t2zv, 01tp87bk1t2zv, ...
8     28198  [01tp87bk1t2zv, 01tp87bk1t2zv, 01tp87bk1t2zv, ...
9     28199  [01tp87bk1t2zv, 01tp87bk1t2zv, 01tp87bk1t2zv, ...
10    28200  [06g9mhm5ba7tt, 0kcbwucxmazcp, 0kkhhb2w93cx0, ...
11    28201  [06g9mhm5ba7tt, 09vrdx888wvvb, 0kcbwucxmazcp, ...
12    28202  [03ggjrmy0wa1w, 06dymzb481vnd, 0aq14dznn91rg, ...
13    28

### Data Ordering

Sorting of datasets in order of SNAP_ID.

In [47]:
df.sort_index(ascending=True,inplace=True)
print(df.shape)
print(df.head(100))

(820, 2)
    SNAP_ID                                             SQL_ID
0     28190  [03ggjrmy0wa1w, 06dymzb481vnd, 0aq14dznn91rg, ...
1     28191  [04kug40zbu4dm, 0a08ug2qc1j82, 0a08ug2qc1j82, ...
2     28192  [01tp87bk1t2zv, 01tp87bk1t2zv, 01tp87bk1t2zv, ...
3     28193  [01tp87bk1t2zv, 01tp87bk1t2zv, 01tp87bk1t2zv, ...
4     28194  [01tp87bk1t2zv, 01tp87bk1t2zv, 01tp87bk1t2zv, ...
5     28195  [06g9mhm5ba7tt, 0kcbwucxmazcp, 0kkhhb2w93cx0, ...
6     28196  [06g9mhm5ba7tt, 0kcbwucxmazcp, 0kkhhb2w93cx0, ...
7     28197  [01tp87bk1t2zv, 01tp87bk1t2zv, 01tp87bk1t2zv, ...
8     28198  [01tp87bk1t2zv, 01tp87bk1t2zv, 01tp87bk1t2zv, ...
9     28199  [01tp87bk1t2zv, 01tp87bk1t2zv, 01tp87bk1t2zv, ...
10    28200  [06g9mhm5ba7tt, 0kcbwucxmazcp, 0kkhhb2w93cx0, ...
11    28201  [06g9mhm5ba7tt, 09vrdx888wvvb, 0kcbwucxmazcp, ...
12    28202  [03ggjrmy0wa1w, 06dymzb481vnd, 0aq14dznn91rg, ...
13    28203  [01tp87bk1t2zv, 01tp87bk1t2zv, 01tp87bk1t2zv, ...
14    28204  [06dymzb481vnd, 0y080mnfaqk3u, 0y

### Univariate Selection

This sextion treats the dataset as a univariate dataset. Therefore the SNAP_ID pertaining to each set of SQL_IDs is removed, with the intent of future classifiers training solely on past SQL executions

In [48]:
print(df.shape)
del df['SNAP_ID']
print(df.shape)

(820, 2)
(820, 1)


### Label Encoding

Since this experiment deals with prediction of upcoming SQL_IDs, respectice SQL_ID strings need to labelled as a numeric representation. Label Encoder will be used here to convert SQL_ID's into a numeric format, which are in turn used for training. Evaluation (achieved predictions) is done so also in numeric format, at which point the label encoder is eventually used to decode back the labels into the original, respetive SQL_ID representation.

This section of the experiment additionally converts the targetted label into a binarized version of the previous achieved categorical numeric values.

* https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/
* https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
* https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelBinarizer.html

NB: Since this experiment is solely focussed on Random Forest Training, One-Hot Encoding will not be used as recommended below:
* https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/

In [49]:
class LabelEncoder:
    """
    Scikit Label Encoder was acting up with the following error whilst using the transform function, even though I tripled 
    checked that the passed data was exactly the same as the one used for training:
    
    * https://stackoverflow.com/questions/46288517/getting-valueerror-y-contains-new-labels-when-using-scikit-learns-labelencoder
    
    So I have rebuilt a similar functionality to categorize my data into numeric digits, as the LabelEncoder is supposed to do.
    """
    #
    def __init__(self):
        self.__class_map = {}
        self.__integer_counter = 0
    #
    def fit(self, X):
        """
        :param - X: python list
        """
        for val in X:
            if val not in self.__class_map:
                self.__class_map[val] = self.__integer_counter
                self.__integer_counter += 1
    #
    def transform(self, X):
        """
        param - X: python list
        """
        encoded_map = []
        for val in X:
            if val in self.__class_map:
                value = self.__class_map[val]
                encoded_map.append(value)
            else:
                raise ValueError('Label Mismatch - Encountered a label which was not trained on.')
        return encoded_map
    #
    def get_class_map(self):
        """
        Returns original classes as a list
        """
        class_map = []
        for key, value in self.__class_map.items():
            class_map.append(key)
        return class_map
    #
    def get_encoded_map(self):
        """
        Returns class encodings as a list
        """
        encoded_map = []
        for key, value in self.__class_map.items():
            encoded_map.append(value)
        return encoded_map

In [50]:
print(df.shape)
print(df.head(10))
le = LabelEncoder()
for index, row in df.iterrows():
    sql_id_list = row['SQL_ID']
    le.fit(sql_id_list)
for index, row in df.iterrows():
    sql_id_list = row['SQL_ID']
    transformed_list = le.transform(sql_id_list)
    df['SQL_ID'].iloc[index] = transformed_list 
#
print("\n----------------------------------\n\nAvailable Classes:")
print('Total SQL_ID Classes: ' + str(len(le.get_class_map())))
print(le.get_class_map()[:10])
print(df.shape)
print(df.head(10))

(820, 1)
                                              SQL_ID
0  [03ggjrmy0wa1w, 06dymzb481vnd, 0aq14dznn91rg, ...
1  [04kug40zbu4dm, 0a08ug2qc1j82, 0a08ug2qc1j82, ...
2  [01tp87bk1t2zv, 01tp87bk1t2zv, 01tp87bk1t2zv, ...
3  [01tp87bk1t2zv, 01tp87bk1t2zv, 01tp87bk1t2zv, ...
4  [01tp87bk1t2zv, 01tp87bk1t2zv, 01tp87bk1t2zv, ...
5  [06g9mhm5ba7tt, 0kcbwucxmazcp, 0kkhhb2w93cx0, ...
6  [06g9mhm5ba7tt, 0kcbwucxmazcp, 0kkhhb2w93cx0, ...
7  [01tp87bk1t2zv, 01tp87bk1t2zv, 01tp87bk1t2zv, ...
8  [01tp87bk1t2zv, 01tp87bk1t2zv, 01tp87bk1t2zv, ...
9  [01tp87bk1t2zv, 01tp87bk1t2zv, 01tp87bk1t2zv, ...

----------------------------------

Available Classes:
Total SQL_ID Classes: 1046
['03ggjrmy0wa1w', '06dymzb481vnd', '0aq14dznn91rg', '0f60bzgt9127c', '0ga8vk4nftz45', '13a9r2xkx1bxb', '13ys8ux8xvrbm', '14f5ngrj3cc5h', '1jhyrdp21f2q6', '1p5grz1gs7fjq']
(820, 1)
                                              SQL_ID
0  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
1  [71, 72, 72, 73, 73, 73, 74, 75, 76,

### Data Normalization

A note regarding normalization. Normalization for this experiment was purposely skipped, since value dimensionality & size is not as important for RandomForest based models. The purity split  does not benefit greatly from such a process:

* https://stats.stackexchange.com/questions/57010/is-it-essential-to-do-normalization-for-svm-and-random-forest
* https://stackoverflow.com/questions/8961586/do-i-need-to-normalize-or-scale-data-for-randomforest-r-package
* https://bmcbioinformatics.biomedcentral.com/track/pdf/10.1186/1471-2105-8-25

### Feature Padding

Since there isn't a fixed number of SQL_ID's per SNAP_ID, each set of SQL_IDs need to be padded so as to assume an equal number if SQL_IDs for the purpose of model fitting.

In [51]:
print("Length at index 0: " + str(len(df['SQL_ID'].iloc[0])))
print(df['SQL_ID'].iloc[0])
print("Length at index 1: " + str(len(df['SQL_ID'].iloc[1])))
print(df['SQL_ID'].iloc[1])
print("Length at index 2: " + str(len(df['SQL_ID'].iloc[2])))
print(df['SQL_ID'].iloc[2])
#
# Retrieve largest length
def pad_datamatrix(df):
    """
    Iterates over dataframe and pads SQL_ID lists accordingly with -1 values
    """
    row_sizes = []
    for index, row in df.iterrows():
        row_sizes.append(len(row['SQL_ID']))
    max_row_size = max(row_sizes)
    #
    # Pad Dataframe Values
    i = 0
    for index, row in df.iterrows():
        length = len(row['SQL_ID'])
        diff = max_row_size - length
        if diff != 0:
            for j in range(length, max_row_size):
                df['SQL_ID'].iloc[i] = np.append(df['SQL_ID'].iloc[i], -1) # Appends -1 to padded values
        # print("Length at index " + str(i) + ": " + str(df['SQL_ID'].iloc[i].size))
        i += 1
    return df
#
df = pad_datamatrix(df)
#
print('\n\n------------------------------------------\n\n')
print("Length at index 0: " + str(len(df['SQL_ID'].iloc[0])))
print(df['SQL_ID'].iloc[0])
print("Length at index 1: " + str(len(df['SQL_ID'].iloc[1])))
print(df['SQL_ID'].iloc[1])
print("Length at index 2: " + str(len(df['SQL_ID'].iloc[2])))
print(df['SQL_ID'].iloc[2])

Length at index 0: 80
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 52, 52, 52, 52, 53, 54, 55, 55, 55, 56, 57, 58, 59, 59, 59, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70]
Length at index 1: 81
[71, 72, 72, 73, 73, 73, 74, 75, 76, 77, 78, 9, 79, 80, 81, 82, 83, 84, 85, 86, 24, 87, 88, 88, 89, 90, 91, 92, 93, 93, 94, 95, 95, 96, 97, 98, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 108, 109, 110, 111, 53, 112, 113, 114, 115, 115, 116, 117, 118, 119, 119, 119, 119, 120, 120, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 132, 133]
Length at index 2: 91
[134, 134, 134, 1, 73, 73, 73, 135, 75, 76, 78, 9, 79, 136, 136, 83, 85, 24, 88, 88, 89, 91, 92, 93, 93, 94, 95, 95, 96, 97, 100, 101, 137, 138, 139, 139, 139, 102, 103, 140, 141, 104, 105, 142, 143, 144, 107, 108, 108, 109, 110, 52, 52, 52, 52, 52, 53, 112

KeyboardInterrupt: 

### Time Series Shifting

Shifting the datasets N lag minutes, in order to transform the problem into a supervised dataset. Each Lag Shift equates to 60 seconds (due to the way design of the data capturing tool). For each denoted lag amount, the same number of feature vectors will be stripped away at the beginning.

Features and Labels are separated into seperate dataframes at this point.

https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = data
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    if n_in != 0:
        for i in range(n_in, 0, -1):
            cols.append(df.shift(i))
            names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    n_out += 1
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
#
def remove_n_time_steps(data, n=1):
    if n == 0:
        return data
    df = data
    headers = df.columns
    dropped_headers = []
    #
    for i in range(1,n+1):
        for header in headers:
            if "(t+"+str(i)+")" in header:
                dropped_headers.append(str(header))
    #
    return df.drop(dropped_headers, axis=1) 
#
# Frame as supervised learning set
shifted_df = series_to_supervised(df, lag, lag)
#
# Seperate labels from features
y_row = []
for i in range(lag+1,(lag*2)+2):
    y_df_column_names = shifted_df.columns[len(df.columns)*i:len(df.columns)*i + 1]
    y_row.append(y_df_column_names)
y_df_column_names = []   
for row in y_row:
    for val in row:
        y_df_column_names.append(val)
#
# y_df_column_names = shifted_df.columns[len(df.columns)*lag:len(df.columns)*lag + len(y_label)]
y_df = shifted_df[y_df_column_names]
X_df = shifted_df.drop(columns=y_df_column_names)
print('\n-------------\nFeatures')
print(X_df.columns)
print(X_df.shape)
print('\n-------------\nLabels')
print(y_df.columns)
print(y_df.shape)
#
# Delete middle timesteps
X_df = remove_n_time_steps(data=X_df, n=lag)
print('\n-------------\nFeatures After Time Shift')
print(X_df.columns)
print(X_df.shape)
# y_df = remove_n_time_steps(data=y_df, n=lag)
print('\n-------------\nLabels After Time Shift')
print(y_df.columns)
print(y_df.shape)

### Expand Feature Lists

Expand Feature Lists, where in each list element is represented as it's own features. Total feature count here equates as follows:

* Features = (lag * SQL_ID per SNAP_ID count) + SQL_ID per SNAP_ID count
* Labels = lag * SQL_ID per SNAP_ID count

In [ ]:
def sequence2features(df):
    """
    Converts pandas sequences into full fledged columns/features
    """
    feature_count = len(df[df.columns[0]].iloc[0])
    for column_name in df.columns:
        data_matrix = []
        new_values = df[column_name].values
        #
        new_values = np.stack(new_values, axis=0 )
        #
        for i in range(1,feature_count+1):
            new_column_name = column_name + "_"+str(i)
            df[new_column_name] = new_values[:,i-1]
        #
        # Drop original list columns
        df.drop(column_name, inplace=True, axis=1)
    return df
#
print('Features')
print('Before: ' + str(X_df.shape))
X_df = sequence2features(df=X_df)
print('After: ' + str(X_df.shape))
#
print('Labels')
print('Before: ' + str(y_df.shape))
y_df = sequence2features(df=y_df)
print('After: ' + str(y_df.shape))

### Feature Selection

Removing Null Columns - this check is redundant, but it double checks that there are no useless/flatline columns

In [ ]:
print('Before: ' + str(df.shape))
#
def drop_flatline_columns(df):
    columns = df.columns
    flatline_features = []
    for i in range(len(columns)):
        try:
            std = df[columns[i]].std()
            if std == 0:
                flatline_features.append(columns[i])
        except:
            pass
    #
    #print('Features which are considered flatline:\n')
    #for col in flatline_features:
    #    print(col)
    print('\nShape before changes: [' + str(df.shape) + ']')
    df = df.drop(columns=flatline_features)
    print('Shape after changes: [' + str(df.shape) + ']')
    print('Dropped a total [' + str(len(flatline_features)) + ']')
    return df
#
df = drop_flatline_columns(df=df)
print('After: ' + str(df.shape))

## Tree Based Model (Many to Many Approach)

### RandomForest Classification (Many To Many)

Classification attemps using RFC - https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

Model Input - Takes training data in the form of past SQL_ID sequences, and trains on a number of past sequence histories, determined by the lag value

Model Output - Outpus future SQL_ID sequences, determined by the lag output

In [ ]:
#
# Random Forest
class RandomForest:
    """
    Random Forest Class (Regression + Classification)
    """
    #
    def __init__(self, n_estimators, max_depth=None, criterion='gini', parallel_degree=1):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.parallel_degree=parallel_degree
        self.criterion = criterion
        self.model = RandomForestClassifier(max_depth=self.max_depth,
                                            n_estimators=self.n_estimators,
                                            criterion=self.criterion,
                                            n_jobs=self.parallel_degree)
    #
    def fit_model(self, X, y):
        """
        Fits training data to target labels
        """
        self.model.fit(X,y)
        print(self.model)
    #
    def predict(self, X):
        yhat = self.model.predict(X)
        return yhat
    #
    def predict_and_evaluate(self, X, y, y_labels, lag, plot=False):
        """
        Runs test data through previously trained model, and evaluate differently depending if a regression of classification model
        """
        yhat = self.predict(X)
        #
        # F1-Score Evaluation
        feature_count = int(y.shape[1] / lag)
        acc_score, f_score = .0,.0
        counter = 0
        for i in range(y.shape[1]):
            acc = accuracy_score(y[:,i], yhat[:,i])
            acc_score += acc
            f1 = f1_score(y[:,i], yhat[:,i], average='micro') 
            f_score += f1
            counter += 1
            if i % (feature_count-1) == 0 and i != 0:
                acc_score = acc_score/feature_count # Averaging accuracy accross total sql_ids for that particular timestep
                f_score = f_score/feature_count # Averaging f-score accross total sql_ids for that particular timestep
                print('Test Accuracy ' + y_labels[i] + ' with LAG value [' + str(counter) + ']: ' +  str(acc_score))
                print('Test FScore ' + y_labels[i] + ' with LAG value [' + str(counter) + ']: ' +  str(f_score) + "\n")
                counter, acc_score, f_score = 0,.0,.0
    #
    @staticmethod
    def write_results_to_disk(path, iteration, lag, test_split, estimator, score, time_train):
        file_exists = os.path.isfile(path)
        with open(path, 'a') as csvfile:
            headers = ['iteration', 'lag', 'test_split', 'estimator', 'score', 'time_train']
            writer = csv.DictWriter(csvfile, delimiter=',', lineterminator='\n', fieldnames=headers)
            if not file_exists:
                writer.writeheader()  # file doesn't exist yet, write a header
            writer.writerow({'iteration': iteration,
                             'lag': lag,
                             'test_split': test_split,
                             'estimator': estimator,
                             'score': score,
                             'time_train': time_train})

In [ ]:
%%time
y_labels = y_df.columns
X_train, X_validate, y_train, y_validate = train_test_split(X_df, y_df, test_size=test_split)
X_train = X_train.values
y_train = y_train.values
#
print("X_train shape [" + str(X_train.shape) + "] Type - " + str(type(X_train)))
print("y_train shape [" + str(y_train.shape) + "] Type - " + str(type(y_train)))
#
X_validate, X_test, y_validate, y_test = train_test_split(X_validate, y_validate, test_size=.5)
#
X_validate = X_validate.values
y_validate = y_validate.values
print("X_validate shape [" + str(X_validate.shape) + "] Type - " + str(type(X_validate)))
print("y_validate shape [" + str(y_validate.shape) + "] Type - " + str(type(y_validate)))
#
X_test = X_test.values
y_test = y_test.values
print("X_test shape [" + str(X_test.shape) + "] Type - " + str(type(X_test)))
print("y_test shape [" + str(y_test.shape) + "] Type - " + str(type(y_test)) + "\n------------------------------")
#
print(X_train[0:5])
print(y_train[0:5])
print('------------------------------------------------------------')
print(X_validate[0:5])
print(y_validate[0:5])
print('------------------------------------------------------------')
print(X_test[0:5])
print(y_test[0:5])
#
# Deletes pandas frames to conserve RAM space.
del X_df, y_df 
#
# Train on discrete data (Train > Validation)
print('Training + Validation')
model = RandomForest(n_estimators=n_estimators,
                     max_depth=max_depth,
                     criterion=criterion,
                     parallel_degree=parallel_degree)
model.fit_model(X=X_train,
                y=y_train)
model.predict_and_evaluate(X=X_validate,
                           y=y_validate,
                           y_labels=y_labels,
                           lag=lag,
                           plot=True)
#
# Train on discrete data (Train + Validation > Test)
print('\n\nTraining + Testing')
model.fit_model(X=X_validate,
                y=y_validate)
model.predict_and_evaluate(X=X_test,
                           y=y_test,
                           y_labels=y_labels,
                           lag=lag,
                           plot=True)